In [1]:
class State:
    def __init__(self, left, right, time_spent):
        self.left = left  # People on left side
        self.right = right  # People on right side
        self.time_spent = time_spent  # Total time spent so far

    def goalTest(self):
        # Goal: All on right side
        return set(self.right) == {'A', 'M', 'GM', 'GF'} and self.time_spent <= 60

    def moveGen(self):
        moves = []
        all_people = {'A': 5, 'M': 10, 'GM': 20, 'GF': 25}

        if len(self.left) >= 2:
            # Send 2 people from left to right
            for i in range(len(self.left)):
                for j in range(i+1, len(self.left)):
                    p1 = self.left[i]
                    p2 = self.left[j]
                    new_left = self.left.copy()
                    new_left.remove(p1)
                    new_left.remove(p2)
                    new_right = self.right + [p1, p2]
                    time = max(all_people[p1], all_people[p2])
                    new_node = Node(new_left, new_right, self.time_spent + time)
                    moves.append((new_node, (p1, p2, '->', time)))

        elif len(self.right) >= 1:
            # Return 1 person from right to left with umbrella
            for i in range(len(self.right)):
                p1 = self.right[i]
                new_right = self.right.copy()
                new_right.remove(p1)
                new_left = self.left + [p1]
                time = all_people[p1]
                new_node = Node(new_left, new_right, self.time_spent + time)
                moves.append((new_node, (p1, '<-', time)))

        return moves

    def __str__(self):
        return f"L: {self.left} | R: {self.right} | Time: {self.time_spent}"

    def __eq__(self, other):
        return set(self.left) == set(other.left) and set(self.right) == set(other.right) and self.time_spent == other.time_spent

    def __hash__(self):
        return hash((frozenset(self.left), frozenset(self.right), self.time_spent))


def reconstructPath(goal_node_pair, CLOSED):
    parent_map = {}
    for node, parent, move in CLOSED:
        parent_map[node] = (parent, move)

    path = []
    goal_node, parent, move = goal_node_pair
    path.append((goal_node, move))
    while parent is not None:
        parent, move = parent_map[parent]
        path.append((parent, move))
    path.reverse()
    return path


def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent, move in OPEN]
    closed_nodes = [node for node, parent, move in CLOSED]
    new_nodes = [c for c, move in children if c not in open_nodes and c not in closed_nodes]
    return [(c, move) for c, move in children if c in new_nodes]


def bfs(start):
    OPEN = [(start, None, None)]
    CLOSED = []

    while OPEN:
        node_tuple = OPEN.pop(0)
        N, parent, move = node_tuple

        if N.goalTest():
            print("Goal Found (BFS) in", N.time_spent, "minutes")
            path = reconstructPath(node_tuple, CLOSED)
            for state, m in path:
                print(state, " via ", m)
            return

        CLOSED.append(node_tuple)
        children = N.moveGen()
        new_children = removeSeen(children, OPEN, CLOSED)
        OPEN += [(c, N, m) for c, m in new_children]

    print("No solution found by BFS")


def dfs(start):
    OPEN = [(start, None, None)]
    CLOSED = []

    while OPEN:
        node_tuple = OPEN.pop(0)
        N, parent, move = node_tuple

        if N.goalTest():
            print("Goal Found (DFS) in", N.time_spent, "minutes")
            path = reconstructPath(node_tuple, CLOSED)
            for state, m in path:
                print(state, " via ", m)
            return

        CLOSED.append(node_tuple)
        children = N.moveGen()
        new_children = removeSeen(children, OPEN, CLOSED)
        OPEN = [(c, N, m) for c, m in new_children] + OPEN

    print("No solution found by DFS")


# Initial State
start = State(['A', 'M', 'GM', 'GF'], [], 0)

print("==== BFS ====")
bfs(start)

print("\n==== DFS ====")
dfs(start)


==== BFS ====


NameError: name 'Node' is not defined

In [ ]:
class State:
    def __init__(self, state):
        self.state = state  # state is a list of characters, e.g. ['E', 'E', 'E', '_', 'W', 'W', 'W']

    def goalTest(self):
        return self.state == ['W', 'W', 'W', '_', 'E', 'E', 'E']

    def moveGen(self):
        children = []
        i = self.state.index('_')  # Find blank position

        def swap_and_create(j):
            new_state = self.state.copy()
            new_state[i], new_state[j] = new_state[j], new_state[i]
            return Node(new_state)

        # Move from left (E) to right
        if i - 1 >= 0 and self.state[i - 1] == 'E':
            children.append(swap_and_create(i - 1))
        if i - 2 >= 0 and self.state[i - 2] == 'E':
            children.append(swap_and_create(i - 2))

        # Move from right (W) to left
        if i + 1 < len(self.state) and self.state[i + 1] == 'W':
            children.append(swap_and_create(i + 1))
        if i + 2 < len(self.state) and self.state[i + 2] == 'W':
            children.append(swap_and_create(i + 2))

        return children

    def __str__(self):
        return ''.join(self.state)

    def __eq__(self, other):
        return self.state == other.state

    def __hash__(self):
        return hash(tuple(self.state))


def reconstructPath(goal_node_pair, CLOSED):
    parent_map = {}
    for node, parent in CLOSED:
        parent_map[node] = parent

    path = []
    goal_node, parent = goal_node_pair
    path.append(goal_node)
    while parent is not None:
        path.append(parent)
        parent = parent_map[parent]

    return path


def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent in OPEN]
    closed_nodes = [node for node, parent in CLOSED]
    new_nodes = [c for c in children if c not in open_nodes and c not in closed_nodes]
    return new_nodes


def bfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair

        if N.goalTest():
            print("Goal is found (BFS):")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node)
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = OPEN + new_pairs
    print("No solution found by BFS")
    return []


def dfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair

        if N.goalTest():
            print("Goal is found (DFS):")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node)
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = new_pairs + OPEN
    print("No solution found by DFS")
    return []


# Initial State: 3 East-bound rabbits, 3 West-bound rabbits, and 1 blank space in middle
start = State(['E', 'E', 'E', '_', 'W', 'W', 'W'])

print("BFS Solution:")
bfs(start)

print("\nDFS Solution:")
dfs(start)
